<a href="https://colab.research.google.com/github/anweshaban21/finoloai-ml/blob/main/GrowthActivistAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

stocks_in_sector = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA'] #same sector companies


pe_ratios = []

for symbol in stocks_in_sector:
    ticker = yf.Ticker(symbol)
    try:
        pe = ticker.info.get('trailingPE')  # trailing PE
        if pe is not None:
            pe_ratios.append(pe)
            print(f"{symbol}: PE = {pe}")
        else:
            print(f"{symbol}: No PE data")
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

# STEP 3: Calculate Industry/Sector PE
if pe_ratios:
    industry_pe = sum(pe_ratios) / len(pe_ratios)
    print(f"\nEstimated Industry PE: {industry_pe:.2f}")
else:
    print("No valid PE data found!")

#stock_data.industry_pe = industry_pe

AAPL: PE = 31.266665
MSFT: PE = 29.683615
GOOGL: PE = 18.800995
AMZN: PE = 31.269928
META: PE = 21.008797
NVDA: PE = 34.52041

Estimated Industry PE: 27.76


In [ ]:
import yfinance as yf
def load_stock_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    stock_data = {
        'pe_ratio': info.get('trailingPE', None),
        'industry_pe': industry_pe,  # We will manually set or estimate it
        'debt_to_equity': info.get('debtToEquity', None),
        'roe': info.get('returnOnEquity', None),
        'operating_margin': info.get('operatingMargins', None),
        'insider_ownership': info.get('heldPercentInsiders', 0) * 100,  # Convert to percentage
        'management_change_recent': False,  # Complex to automate (optional for now)
    }

    return stock_data

In [ ]:
class ActivistGrowthAgent:
    def __init__(self, stock_data):
        self.data = stock_data

    def evaluate(self):
        score = 0

        # P/E relative to industry (Assume industry average is 20 for now)
        #industry_pe = self.data.get('industry_pe') or 20

        if self.data['pe_ratio'] is not None and self.data['pe_ratio'] < industry_pe:
            score += 2

        # Debt manageable
        if self.data['debt_to_equity'] is not None and self.data['debt_to_equity'] < 200:
            score += 2

        # Good ROE
        if self.data['roe'] is not None and self.data['roe'] > 0.10:
            score += 1

        # Good Operating Margin
        if self.data['operating_margin'] is not None and self.data['operating_margin'] > 0.15:
            score += 1

        # Insider Ownership > 5%
        if self.data['insider_ownership'] > 5:
            score += 1

        # Management Change (optional, for now False)
        if self.data['management_change_recent']:
            score += 2

        if score >= 7:
            return "High Potential Growth"
        elif 4 <= score < 7:
            return "Moderate Potential"
        else:
            return "Low Potential"

# Step 3: Test it!
ticker = "AMZN"  # Try Apple for example
stock_data = load_stock_data(ticker)

# (Optional) Add missing manual data
stock_data['industry_pe'] = industry_pe  # You can lookup or use approx values
stock_data['management_change_recent'] = False  # You can make a smarter module later

agent = ActivistGrowthAgent(stock_data)
prediction = agent.evaluate()

print(f"Prediction for {ticker}: {prediction}")

Prediction for AMZN: Moderate Potential
